# Introduction

Name: Cody Lam

I understand that my submission needs to be my own work: C.L.

I understand that ChatGPT / Copilot / other AI tools are not allowed: C.L.

Total Points: **75**

Complete this notebook and submit it (save/print as pdf). Make sure all output is correct in the pdf before submitting (it sometimes gets cut off). The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Z3 and Z3py resources

For this problem you will be solving constraint satisfaction and logic problems using the Z3 SMT solver via the z3py python library.

The demo code using z3py we went over in class is here: [link](https://drive.google.com/drive/folders/16HQXiwdcaman1IpC7H6vSJ71QQQ-aL2z?usp=sharing)

Online help for z3py is here: [link](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## Installation

We first install to get the z3solver library using pip and test that works.

In [2]:
!pip install z3-solver

In [3]:
# Run the first example from the z3py guide as a test: https://ericpony.github.io/z3py-tutorial/guide-examples.htm
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


# Sudoku as a constraint satisfaction problem (CSP)

Sudoku is a popular number-placement puzzle that originated in France in the end of the 19th century.  Modern Sudoku was likely invented by Howard Garns from Connersville, Indiana and was first published in 1979 under the name *Number Place*. The objective of the puzzle is to place numbers 1-9 on a 9$\times$9 grid, such that each number occurs only once in every row, every column, and every of the nine 3$\times$3 sub-grids that compose the main grid.
Sudoku puzzles are grids that have been partially occupied with numbers. The task is then to occupy the remaining fields in such a way that the constraints on rows, columns, and sub-grids are satisfied.
For more information about Sudoku refer to its Wikipedia page at http://en.wikipedia.org/wiki/Sudoku.

This problem has two parts. In the first part, you will write the boolean constraints in mathematical notation for solving a Sudoku puzzle. In the second part, you will write code and invoke the Z3 solver to solve the Sudoku instance and answer various questions about the solution.

## Part 1 (Constraints)
In text, define and write constraints over **boolean** variables corresponding to each number being in each cell. For example, you may use $X_{i,j}^k$ as teh variable that is true if and only if the number $k$ is in row $i$ and column $j$ (the variable is true if $k$ is in cell $(i, j)$ and false otherwise). Now write the following boolean constaints over these variables:

* Write the boolean formula for the constraints that the number 5 can occur at most once in the first row. **(3 point)**
* Write the boolean formula for the constraints that the number 6 can occur at most once in the first column. **(3 point)**
* Write the boolean formula for the constraints that the number 9 can occur at most once in the top left 3$\times$3 sub-grid. **(4 point)**

Solution:

## Part 2 (Coding)

Encode the above constraints and all the other ones as a CSP using z3py and solve the Sudoku instance given in the figure below. **Use only boolean variables** and do not use the **Distinct** function.

* Provide the code to solve that solves the given problem instance using z3py and only boolean variables. The code should output a reasonable visualization of the solution, for example printed in text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/sudoku1.jpg" width="400"/>



In [1]:
# z3py code to solve above Sudoku goes here. Do not use Int(*), only Bool(*)
from z3 import *

X = [[[Bool(f'X_{k}_{i}_{j}') for j in range(9)] for i in range(9)] for k in range(1, 10)]

s = Solver()
# Constraint 1: Each cell (i,j) must contain exactly one number
for i in range(9):
    for j in range(9):
        s.add(Or([X[k][i][j] for k in range(9)]))  # At least one number in cell
        for k1 in range(9):
            for k2 in range(k1 + 1, 9):
                s.add(Not(And(X[k1][i][j], X[k2][i][j])))  # No two numbers in the same cell

# Constraint 2: Checking if each number is appearing at most once in a row
for i in range(9):
    for k in range(9):
        for j1 in range(9):
            for j2 in range(j1 + 1, 9):
                s.add(Not(And(X[k][i][j1], X[k][i][j2])))

# Constraint 3: Checks if each number is appearing at most once in a column
for j in range(9):
    for k in range(9):
        for i1 in range(9):
            for i2 in range(i1 + 1, 9):
                s.add(Not(And(X[k][i1][j], X[k][i2][j])))

# Constraint 4: Checks if each number is appearing once in the 3x3 subgrid
for block_i in range(3):
    for block_j in range(3):
        for k in range(9):
            cells = []
            for i in range(3):
                for j in range(3):
                    cells.append(X[k][block_i * 3 + i][block_j * 3 + j])
            for c1 in range(len(cells)):
                for c2 in range(c1 + 1, len(cells)):
                    s.add(Not(And(cells[c1], cells[c2])))

# Initial Sudoku puzzle
puzzle = [
    [0,1,0, 4,0,2, 0,5,0],
    [5,0,0, 0,0,0, 0,0,6],
    [0,0,0, 3,0,1, 0,0,0],
    [7,0,5, 0,0,0, 4,0,8],
    [0,0,0, 0,0,0, 0,0,0],
    [2,0,8, 0,0,0, 5,0,9],
    [0,0,0, 9,0,6, 0,0,0],
    [6,0,0, 0,0,0, 0,0,2],
    [0,7,0, 1,0,3, 0,4,0],
]

# Add puzzle clues
for i in range(9):
    for j in range(9):
        val = puzzle[i][j]
        if val != 0:
            s.add(X[val-1][i][j])  # value val is present at (i,j)

# Solve the puzzle and display it
if s.check() == sat:
  m = s.model()
  solution = [[0 for _ in range(9)] for _ in range(9)]
  for i in range(9):
    for j in range(9):
      for k in range(9):
        if m.evaluate(X[k][i][j]):
          solution[i][j] = k + 1

  # Print the solution
  for i in range(9):
    row = ''
    for j in range(9):
      row += str(solution[i][j]) + ' '
      # When a 3x3 is complete put a '|'
      if j % 3 == 2 and j != 8:
        row += '| '
    print(row)
    # prints out the lines that will separate the rows
    if i % 3 == 2 and i != 8:
      print('-'*21)
else:
  print("No solution was found")


3 1 9 | 4 6 2 | 8 5 7 
5 2 4 | 7 9 8 | 1 3 6 
8 6 7 | 3 5 1 | 9 2 4 
---------------------
7 3 5 | 2 1 9 | 4 6 8 
1 9 6 | 8 4 5 | 2 7 3 
2 4 8 | 6 3 7 | 5 1 9 
---------------------
4 5 3 | 9 2 6 | 7 8 1 
6 8 1 | 5 7 4 | 3 9 2 
9 7 2 | 1 8 3 | 6 4 5 


### Uniqueness
Is your solution unique? Prove it with a Z3py solver or provide a second solution. **(5 points)**

In [2]:
'''
In order to check if the solution is unique, I added the solition into an array called diff
'''
if s.check() == sat:
    m = s.model()
    solution = [[0 for _ in range(9)] for _ in range(9)]
    diff_solution = []  # To exclude this solution
    for i in range(9):
        for j in range(9):
            for k in range(9):
                val = m.evaluate(X[k][i][j])
                if val:
                    solution[i][j] = k + 1
                    diff_solution.append(Not(X[k][i][j]))
    # Print first solution
    print("First solution:")
    for i in range(9):
        row = ''
        for j in range(9):
            row += str(solution[i][j]) + ' '
            if j % 3 == 2 and j != 8:
                row += '| '
        print(row)
        if i % 3 == 2 and i != 8:
            print('-'*21)

    # Check for uniqueness by excluding this solution
    s.push()
    s.add(Or(diff_solution))  # At least one cell must differ
    if s.check() == sat:
        print("\n Another solution exists. The solution is not unique.")
    else:
        print("\n The solution is unique. No other valid solution exists.")
    s.pop()
else:
    print("No solution found.")

First solution:
3 1 9 | 4 6 2 | 8 5 7 
5 2 4 | 7 9 8 | 1 3 6 
8 6 7 | 3 5 1 | 9 2 4 
---------------------
7 3 5 | 2 1 9 | 4 6 8 
1 9 6 | 8 4 5 | 2 7 3 
2 4 8 | 6 3 7 | 5 1 9 
---------------------
4 5 3 | 9 2 6 | 7 8 1 
6 8 1 | 5 7 4 | 3 9 2 
9 7 2 | 1 8 3 | 6 4 5 

 The solution is unique. No other valid solution exists.


### More Uniqueness
If you delete the 1 in the top left box of the Sudoku problem above, how many unique solutions are there? Hint: should be fewer than 5000. **(5 points)**

In [3]:
puzzle2 = [
    [0,0,0, 4,0,2, 0,5,0],
    [5,0,0, 0,0,0, 0,0,6],
    [0,0,0, 3,0,1, 0,0,0],
    [7,0,5, 0,0,0, 4,0,8],
    [0,0,0, 0,0,0, 0,0,0],
    [2,0,8, 0,0,0, 5,0,9],
    [0,0,0, 9,0,6, 0,0,0],
    [6,0,0, 0,0,0, 0,0,2],
    [0,7,0, 1,0,3, 0,4,0],
]

# Add puzzle clues
for i in range(9):
    for j in range(9):
        val = puzzle2[i][j]
        if val != 0:
            s.add(X[val-1][i][j])

# Count solutions
count = 0
max_solutions = 5000

while count < max_solutions and s.check() == sat:
    m = s.model()
    diff = []
    for i in range(9):
        for j in range(9):
            for k in range(9):
                if m.evaluate(X[k][i][j]):
                    diff.append(Not(X[k][i][j]))
    s.add(Or(diff))  # exclude this solution
    count += 1
    if count % 100 == 0:
        print(f"{count} solutions found...")

print(f"\n✅ Total number of unique solutions: {count}")


✅ Total number of unique solutions: 1


# Miracle Sudoku

Using z3py and only boolean variables, solve the Miracle Sudoku ([spoiler](https://www.youtube.com/watch?v=yKf9aUIxdb4)).
In this puzzle normal Sudoku rules apply in addition to the following rules:
* Any two cells separated by a knight's move from chess (moving forward two boxes and over one, in any direction) cannot contain the same digit (see image below).
* Any two cells separated by a king's move from chess (one box in any direction including diagonals) cannot contain the same digit.
* Any two orthogonally adjacent (up, down, left, or right) cells cannot contain consecutive digits.

### Knight's move constraint image
<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/knights.jpg" width="400"/>

The knight's move constraint would mean the yellow boxes cannot contain a 1.


## Miracle Sudoku Starting Information
Provide the code to solve the puzzle given below. The code should output a reasonable visualization of the solution, for example using text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/miracle.jpg" width="400"/>

In [ ]:
# Solution code for Miracle Sudoku using z3py
from z3 import *



---
# Number Challenge in Z3py

Write Z3py code to solve the Number Challenge puzzle shown below and print the solution found. You can use the `Int` type to create integer variables in z3py. **(20 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/chal.jpg" width="300"/>

In [ ]:
# Write your solution code for Number Challenge using z3py
from z3 import *

# example using Int type (feel free to delete)
s = Solver()

x = Int('x')
y = Int('y')

s.add((2*y) + x == 20)
s.add(x - y == 2)

if s.check() == sat:
    print("sat")
    m = s.model()
    print(f"model: {m}")

    print(f"x: {x}")
    print(f"y: {y}")
else:
    print("unsat")

sat
model: [y = 6, x = 8]
x: x
y: y





---


# Propositional Logic

Next, we will use truth tables and z3py to prove logical entailment.

## Truth Tables

Using markdown tables and bold to show relevant rows, use the truth table method to show logical truths.


### Example

Here's an example that shows $p, p \rightarrow q \models q$.  

| p | q | p -> q |
|-------|-------|---------------------|
| **T**     | **T**     | **T**                   |
| T     | F     | F                   |
| F     | T     | T                   |
| F     | F     | T                   |

Since $q$ is true whenever $p$ and $p \rightarrow q$ are true, the expression is true.

### Questions

Prove that the following semantic entailment expressions are true by using the truth table approach.  Bold the appropriate rows **(7 point)**

 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$
 * $\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$

(add markdown tables here)



---


 ## Logical Entailment using z3py

 Use Z3py to prove the above two entailments are true using satisfiability. See section 7.5 in the book for details. **(8 points)**

In [ ]:
# answer code here